https://python.langchain.com/docs/tutorials/agents/  
  
语言模型本身无法采取行动——它们只是输出文本。LangChain的一个重要用例是创建代理。代理是利用LLM作为推理引擎来决定需要采取哪些行动及执行这些行动所需的输入的系统。在执行这些行动后，结果可以反馈到LLM中以确定是否需要更多的行动，或者是否可以结束。这通常是通过工具调用来实现的。  
  
在这个教程中，我们将构建一个能够与搜索引擎交互的代理。你将能够向这个代理提问，观察它调用搜索工具，并与它进行对话。  

端到端代理  
  
下面的代码片段展示了一个功能齐全的代理，它使用LLM来决定使用哪些工具。该代理配备了一个通用的搜索工具，并且具有对话记忆——这意味着它可以作为多轮聊天机器人使用。  
  
在本指南的其余部分中，我们将详细介绍各个组件及其功能——但如果你只是想获取一些代码并开始使用，下面是快速示例！  

In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var]=getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")

In [3]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(
    #model_name="deepseek-r1:32b",
    model_name="qwen2",
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=False
)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# 初始化 Hugging Face 嵌入模型
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_107861/127560303.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_107861/127560303.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/haoyu/anaconda3/envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook 

In [5]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [6]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hello Bob! How can I assist you today?


In [7]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_5yadooaw)
 Call ID: call_5yadooaw
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in san francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741586700, 'localtime': '2025-03-09 23:05'}, 'current': {'last_updated_epoch': 1741586400, 'last_updated': '2025-03-09 23:00', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wi

我们首先需要创建我们想要使用的工具。我们的主要选择将是Tavily——一个搜索引擎。LangChain中有一个内置的工具，可以让我们轻松地将Tavily搜索引擎用作工具。

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741584916, 'localtime': '2025-03-09 22:35'}, 'current': {'last_updated_epoch': 1741584600, 'last_updated': '2025-03-09 22:30', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 282, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 74, 'cloud': 50, 'feelslike_c': 10.5, 'feelslike_f': 50.8, 'windchill_c': 9.0, 'windchill_f': 48.3, 'heatindex_c': 9.3, 'heatindex_f': 48.7, 'dewpoint_c': 8.8, 'dewpoint_f': 47.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 6.9, 'gust_kph': 11.1}}", 'score': 

接下来，让我们学习如何使用语言模型来调用工具。LangChain支持许多不同的语言模型，你可以根据需要选择。

In [9]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(
    #model_name="deepseek-r1:32b",
    model_name="qwen2",
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=False
)

你可以通过传入一个消息列表来调用语言模型。默认情况下，响应是一个内容字符串。

In [10]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hello! How can I help you today? Do you have any questions or topics you'd like to discuss? I'm here to provide information and assist with various inquiries. Feel free to ask anything."

我们现在可以看看如何使用这个模型来进行工具调用。为了实现这一点，我们使用.bind_tools来让语言模型了解这些工具。

In [11]:
model_with_tools = model.bind_tools(tools)

我们现在可以调用这个模型了。首先，让我们用一个普通的消息来调用它，看看它是如何响应的。我们可以查看内容字段以及工具调用字段。

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


现在，让我们尝试用一些期望调用工具的输入来调用它。

In [13]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in SF'}, 'id': 'call_fz59xlsj', 'type': 'tool_call'}]


我们可以看到，现在没有文本内容，但是有一个工具调用！它希望我们调用Tavily搜索工具。  
  
这还没有真正调用该工具——它只是告诉我们应该这样做。为了实际调用它，我们需要创建我们的代理。

创建代理  
  
现在我们已经定义了工具和语言模型（LLM），我们可以创建代理了。我们将使用LangGraph来构建这个代理。目前，我们使用高级接口来构建代理，但LangGraph的优点在于这个高级接口背后有一个底层的、高度可控的API，供你在需要时修改代理逻辑。  
  
现在，我们可以用LLM和工具初始化代理。  
  
注意，我们传入的是模型，而不是带有工具的模型（model_with_tools）。这是因为create_react_agent会在内部自动调用.bind_tools来为我们绑定工具。  

In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

运行代理  
  
我们现在可以用一些查询来运行代理了！请注意，目前这些查询都是无状态的（它不会记住之前的交互）。注意，代理在交互结束时会返回最终状态（包括任何输入，我们稍后会看到如何只获取输出）。  
  
首先，让我们看看在不需要调用工具时它是如何响应的：  

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='07667cda-96be-4dc9-bb19-633da41eb411'),
 AIMessage(content='Hi there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 174, 'total_tokens': 185, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-1849265b-3a02-4edc-bc5a-902c3cb1945e-0', usage_metadata={'input_tokens': 174, 'output_tokens': 11, 'total_tokens': 185, 'input_token_details': {}, 'output_token_details': {}})]

现在让我们在一个应该调用工具的例子上尝试一下。

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='d56597cb-5b8e-40af-8ee2-8c1d78ca1011'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i02l9a8q', 'function': {'arguments': '{"query":"weather in sf"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 179, 'total_tokens': 210, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d43806cf-f147-4081-a3a0-d5de43d1f9e4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'call_i02l9a8q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 179, 'output_tokens': 31, 'total_tokens': 210, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='[{"title": "Weath

流式消息  
  
我们已经看到了如何使用.invoke来调用代理以获取最终响应。如果代理执行多个步骤，这可能需要一些时间。为了显示中间进度，我们可以实现在发生时即时流式返回消息。  

In [17]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_u7l7eh86)
 Call ID: call_u7l7eh86
  Args:
    query: weather in sf
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in san francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741583767, 'localtime': '2025-03-09 22:16'}, 'current': {'last_updated_epoch': 1741583700, 'last_updated': '2025-03-09 22:15', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.5, 'win

流式传输令牌  
  
除了流式返回消息之外，流式返回令牌也很有用。我们可以通过指定stream_mode="messages"来实现这一点。

In [18]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The current weather in San Francisco is partly cloudy with a temperature of 11.1°C. The wind speed is 4.5 mph coming from the WNW direction, and there are no reports of precipitation. Expect mild humidity at 74% while cloud coverage reaches about 50%. The comfortable feel outside is similar to a temperature of 10.5 °C with some cloud chill affecting perceived temperatures. Note that the provided data was last updated on March 9th, 2025, and might differ slightly today.

Additionally, for more detailed forecasts and long term weather conditions in San Francisco throughout March 2025, you can check out this resource: [Extended weather forecast](https://world-weather.info/forecast/usa/san_francisco/march-2025/) from world-weather.info.|

添加记忆功能  
  
如前所述，这个代理是无状态的。这意味着它不会记住之前的交互。为了赋予它记忆功能，我们需要传入一个检查点工具（checkpointer）。当传入检查点工具时，在调用代理时还需要传入一个线程ID（thread_id），以便它知道要从哪个线程/对话继续进行。  

In [19]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 176, 'total_tokens': 187, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-7cd17604-e6ac-4efc-b48c-c13a28218a14-0', usage_metadata={'input_tokens': 176, 'output_tokens': 11, 'total_tokens': 187, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Sure, your name is Bob. Is there anything else you need help with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 201, 'total_tokens': 218, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-774af2c5-ab8b-4514-b189-edfb8b3230ae-0', usage_metadata={'input_tokens': 201, 'output_tokens': 17, 'total_tokens': 218, 'input_token_details': {}, 'output_token_details': {}})]}}
----


如果你想开始一个新的对话，你只需要更改使用的线程ID。

In [23]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="It seems like you're asking for your name, but as an AI, I don't have access to personal information such as names unless it has been provided to me in the course of our conversation. If you've given me a name before, then that's what I would remember. If not, could you please provide more context or details?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 177, 'total_tokens': 247, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-4773e0b0-fa6c-46cb-b2e2-bdd9e17c04d3-0', usage_metadata={'input_tokens': 177, 'output_tokens': 70, 'total_tokens': 247, 'input_token_details': {}, 'output_token_details': {}})]}}
----
